# Title

In this notebook, we try to annotate the title of the reference using the previously extracted information. We apply the same paradigm and try to detect the start of the title using the other entities and the end of the title based on chars and additional information.

In [ ]:
%inputDir data/train
%outputDir out_title_train
%displayMode EVALUATION
%evalTypes Title
%writescript ./Title.ruta
%saveTypeSystem ./TitleTypeSystem.xml

TYPESYSTEM ReferencesTypeSystem;
SCRIPT Date;
SCRIPT Author;

// only try to find dates and authors if there aren't any yet
Document{-CONTAINS(Date)-> CALL(Date)};
Document{-CONTAINS(Author)-> CALL(Author)};

INT maxLength = 50;
DECLARE TitleStart, TitleStop, Quote, Quoted;

BLOCK(utils) Reference{}{
    
    // detect quoted text as it specifies the title in some formats
    SPECIAL{REGEXP("\"")->Quote};
    (Quote ANY[1,maxLength]{-PARTOF(Quote)} Quote){-> Quoted};
    
    // the title normally starts after the author or date
    (Author Date?){-> MARKLAST(TitleStart)};
    Reference{-CONTAINS(TitleStart), -CONTAINS(Author)} ->{
        // in case there was no author but an editor or something else 
        (AuthorStop Date?){-> MARKLAST(TitleStart)};
    };
    
    // the title normally stops with a period
    PERIOD{-PARTOF(Author),-PARTOF(Title)-> TitleStop};
    
    // some specific periods are excluded
    NUM ts:@TitleStop{-> UNMARK(ts)} NUM;
    ts:TitleStop{ENDSWITH(Reference)-> UNMARK(ts)};
    ts:TitleStop{-> UNMARK(ts)} # TitleStart;
    W{REGEXP("pp", true)} ts:@TitleStop{-> UNMARK(ts)};
    sw1:SW ts:@TitleStop{sw1.end==ts.begin,ts.end==sw2.begin-> UNMARK(ts)} sw2:SW;
    
    // if we did not find any, maybe it ends with a comma
    Reference{-CONTAINS(TitleStop)} ->{
        TitleStart # COMMA{->TitleStop};
    };
}

BLOCK(Title) Reference{}{
    // annotate titles either based on quoted text or based on start and stop (boundaries)
    TitleStart Quoted{-PARTOF(Title)-> Title};
    TitleStart (ANY[1,maxLength]{-PARTOF(TitleStop),-PARTOF(Title)} TitleStop){-> Title};
    
    // extend the span if there is an period
    t:Title{->t.end=e.end} e:ANY{PARTOF({PERIOD,COMMA})};
    
    // remove duplicates just in case
    t1:Title{CONTAINS(Title,2,100)}->{t2:Title{t2!=t1-> UNMARK(t2)};};
}



In [ ]:
%inputDir out_title_train
%outputDir trash
%displayMode CSV
%csvConfig BadReference

DECLARE BadReference;
Reference{OR(CONTAINS(FalsePositive),CONTAINS(FalseNegative)),-PARTOF(BadReference)-> BadReference};

COLOR(TitleStart, "green");
COLOR(TitleStop, "red");
COLOR(TruePositive, "lightgreen");
COLOR(FalsePositive, "lightblue");
COLOR(FalseNegative, "pink");

In [ ]:
%inputDir data/validation
%outputDir out_title_validation
%displayMode EVALUATION
%evalTypes Title

SCRIPT Title;
CALL(Title);

In [ ]:
%inputDir out_title_validation
%outputDir trash
%displayMode CSV
%csvConfig BadReference

DECLARE BadReference;
Reference{OR(CONTAINS(FalsePositive),CONTAINS(FalseNegative)),-PARTOF(BadReference)-> BadReference};

COLOR(TitleStart, "green");
COLOR(TitleStop, "red");
COLOR(TruePositive, "lightgreen");
COLOR(FalsePositive, "lightblue");
COLOR(FalseNegative, "pink");

In [ ]:
%inputDir data/test
%displayMode EVALUATION
%evalTypes Title

SCRIPT Title;
CALL(Title);